# HVM Flow Curve: NLSQ + NUTS Bayesian Inference

## Introduction

This notebook demonstrates the **full Bayesian inference pipeline** for fitting the
HVM to experimental flow curve data. The workflow follows the RheoJAX standard:
NLSQ point estimation for warm-start, then NumPyro NUTS sampling for uncertainty
quantification.

**Physical context:** At steady state, the E-network stress vanishes ($\sigma_E \to 0$)
because BER continuously redefines the natural reference state. The flow curve is
therefore dominated by the permanent network ($\sigma_P = G_P \gamma$) and the
dissociative network ($\sigma_D = \eta_D \dot\gamma$). This makes the flow curve
ideal for constraining $G_P$, $G_D$, and $k_d^D$ independently.

> **Handbook:** See [HVM Knowledge Extraction](../../docs/source/models/hvm/hvm_knowledge.rst)
> for the recommended multi-protocol fitting strategy. Flow curve fitting is most
> effective when combined with SAOS data for cross-validation.

## Workflow

1. Load experimental data and perform quality checks
2. NLSQ optimization for point estimates
3. Bayesian inference with NUTS for uncertainty quantification
4. Posterior predictive analysis
5. Component decomposition ($\sigma_P$, $\sigma_D$ contributions)

## Prerequisites

- **Notebook 01** (SAOS) or **Notebook 07** (Overview) for HVM architecture
- **Notebook 05** (Flow curve basics) for the $\sigma_E \to 0$ physics

## 1. Setup

In [1]:
# Float64 configuration (CRITICAL: import before JAX)
from rheojax.core.jax_config import safe_import_jax, verify_float64

jax, jnp = safe_import_jax()
verify_float64()

# Standard imports
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

# Add utils to path
sys.path.insert(0, str(Path("..").resolve()))

# Model and utilities
from utils.hvm_data import check_data_quality, load_emulsion_flow_curve
from utils.hvm_fit import (
    FAST_MODE,
    get_bayesian_config,
    get_output_dir,
    posterior_predictive_1d,
    print_convergence,
    print_parameter_table,
    run_nlsq_protocol,
    run_nuts,
    save_figure,
    save_results,
)

from rheojax.models import HVMLocal

print(f"FAST_MODE: {FAST_MODE}")
print(f"JAX version: {jax.__version__}")
import os
import sys

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

FAST_MODE: True
JAX version: 0.8.3


## 2. Load Experimental Data

We load flow curve data from a concentrated oil-water emulsion at φ=0.74. This system exhibits:
- Yield stress plateau at low shear rates (elastic network)
- Shear thinning at intermediate rates
- Newtonian-like behavior at high rates

In [2]:
# Load emulsion data (φ=0.74)
gamma_dot, stress = load_emulsion_flow_curve(phi=0.74)

print(f"Data points: {len(gamma_dot)}")
print(f"Shear rate range: {gamma_dot.min():.2e} to {gamma_dot.max():.2e} 1/s")
print(f"Stress range: {stress.min():.2e} to {stress.max():.2e} Pa")

# Quality check
check_data_quality(gamma_dot, stress, "Emulsion φ=0.74 flow curve")

Data points: 30
Shear rate range: 1.59e-03 to 9.77e+02 1/s
Stress range: 1.32e+01 to 1.92e+02 Pa
Data QC: Emulsion φ=0.74 flow curve
  Points: 30
  x range: [0.001591, 977.1]
  y range: [13.15, 192.4]
  Status: PASSED


{'name': 'Emulsion φ=0.74 flow curve',
 'n_points': 30,
 'x_range': (0.0015905448933224, 977.1455426886162),
 'y_range': (13.153682802281615, 192.38193348443005),
 'has_nan': False,
 'has_inf': False,
 'all_positive_y': True,
 'monotonic_x': True,
 'issues': [],
 'passed': True}

## 3. Data Exploration

Visualize the raw data to identify key features:
- Yield plateau at low γ̇ (indicates permanent network)
- Power-law regime (shear thinning)
- Dynamic range for parameter inference

In [3]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(gamma_dot, stress, 'ko', markersize=6, label='Emulsion φ=0.74')
ax.set_xlabel('Shear Rate γ̇ (1/s)', fontsize=12)
ax.set_ylabel('Stress σ (Pa)', fontsize=12)
ax.set_title('Emulsion Flow Curve: Raw Data', fontsize=13, weight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3, which='both', ls='-', lw=0.5)
plt.tight_layout()
display(fig)
plt.close(fig)

<Figure size 800x500 with 1 Axes>

## 4. Model Setup

Initialize HVMLocal with physically reasonable parameter bounds for emulsion systems.

**Key parameters for flow curves:**
- **G_P**: Permanent network modulus (yield plateau)
- **G_E**: Exchangeable network modulus (vitrimer contribution)
- **G_D**: Dissociative network modulus (viscous flow)
- **k_d_D**: Dissociation rate (controls viscosity)

In [4]:
# Initialize model with dissociative network
model = HVMLocal(include_dissociative=True, kinetics='stress')

# Set initial parameter values appropriate for emulsion
model.parameters.set_value("G_P", 10.0)      # Permanent network (elastic plateau)
model.parameters.set_value("G_E", 5.0)       # Exchangeable network
model.parameters.set_value("G_D", 2.0)       # Dissociative network (viscous)
model.parameters.set_value("nu_0", 1e10)     # Exchange frequency
model.parameters.set_value("E_a", 80e3)      # Activation energy
model.parameters.set_value("V_act", 1e-5)   # Activation volume (m³)
model.parameters.set_value("k_d_D", 1.0)     # Dissociation rate
model.parameters.set_value("T", 300.0)       # Temperature (K)

print("Model initialized with parameters:")
print(model.parameters)

Model initialized with parameters:


## 5. NLSQ Fit

Perform non-linear least squares optimization using NLSQ (5-270x faster than scipy).

**Strategy:**
- Use log-space residuals for better dynamic range
- Warm-start from initial values above
- Check convergence and R² metric

In [5]:
# Run NLSQ optimization
print("Running NLSQ optimization...")
nlsq_vals = run_nlsq_protocol(
    model,
    gamma_dot,
    stress,
    test_mode='flow_curve',
    use_log_residuals=True
)

print("\nNLSQ optimization complete.")
print(f"Fitted parameters: {nlsq_vals}")
print(f"\nModel R²: {(model._nlsq_result.r_squared or 0):.4f}")

INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=8 | loss=linear | ftol=1.0000e-06 | xtol=1.0000e-06 | gtol=1.0000e-06


Running NLSQ optimization...


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=1.449159s


INFO:nlsq.least_squares:Convergence reason=`xtol` termination condition is satisfied. | iterations=1 | final_cost=34.0555 | elapsed=1.449s | final_gradient_norm=9.7433e+07



NLSQ optimization complete.
Fitted parameters: {'G_P': 10.0, 'G_E': 5.0, 'nu_0': 10000000000.0, 'E_a': 80000.0, 'V_act': 1e-05, 'T': 300.0, 'G_D': 6.770575346307817, 'k_d_D': 0.9999999045885866}

Model R²: 0.0000


## 6. Plot NLSQ Fit with Component Decomposition

Visualize the total fit and decompose contributions from:
- **σ_P**: Permanent network (elastic/yield plateau)
- **σ_D**: Dissociative network (viscous flow)

Note: σ_E → 0 at steady state (natural state tracks deformation).

In [6]:
# Generate dense prediction grid
gamma_dot_fit = np.logspace(np.log10(gamma_dot.min()), np.log10(gamma_dot.max()), 200)

# Get total stress prediction
stress_fit = model.predict(gamma_dot_fit, test_mode='flow_curve')

# Plot
fig, ax = plt.subplots(figsize=(9, 6))

# Data
ax.loglog(gamma_dot, stress, 'ko', markersize=7, label='Data', zorder=3)

# NLSQ fit
ax.loglog(gamma_dot_fit, stress_fit, 'r-', lw=2.5, label='HVM NLSQ fit')

ax.set_xlabel("Shear rate (1/s)", fontsize=12)
ax.set_ylabel("Stress (Pa)", fontsize=12)
ax.set_title("HVM Flow Curve: NLSQ Fit", fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
save_figure(fig, get_output_dir('flow_curve'), 'hvm_flow_curve_nlsq_fit.png')
plt.close(fig)

Figure saved: /Users/b80985/Projects/rheojax/examples/outputs/hvm/flow_curve/figures/hvm_flow_curve_nlsq_fit.png


## 7. Bayesian Inference with NUTS

Perform Bayesian inference using NumPyro NUTS sampler with NLSQ warm-start.

**Configuration:**
- 4 chains for production-grade diagnostics
- NLSQ values as warm-start (2-5x speedup)
- ArviZ diagnostics (R-hat, ESS, BFMI)

Target metrics:
- R-hat < 1.01 (convergence)
- ESS > 400 (effective samples)
- No divergences

In [7]:
# Get Bayesian configuration (respects FAST_MODE)
bayes_config = get_bayesian_config()
print(f"Bayesian config: {bayes_config}")

# Run NUTS
print("\nRunning Bayesian inference (NUTS)...")
result = run_nuts(
    model,
    gamma_dot,
    stress,
    test_mode='flow_curve',
    **bayes_config
)

# Define parameter names for diagnostics
param_names = list(model.parameters.keys())

print("\nBayesian inference complete.")

Bayesian config: {'num_warmup': 50, 'num_samples': 100, 'num_chains': 1}

Running Bayesian inference (NUTS)...


  0%|          | 0/150 [00:00<?, ?it/s]

warmup:   1%|          | 1/150 [00:00<02:08,  1.16it/s, 1 steps of size 1.95e+00. acc. prob=0.00]

sample:  40%|████      | 60/150 [00:00<00:01, 84.93it/s, 63 steps of size 1.33e-02. acc. prob=0.92]

sample:  68%|██████▊   | 102/150 [00:01<00:00, 141.50it/s, 63 steps of size 1.33e-02. acc. prob=0.94]

sample:  93%|█████████▎| 140/150 [00:01<00:00, 185.39it/s, 63 steps of size 1.33e-02. acc. prob=0.93]

sample: 100%|██████████| 150/150 [00:01<00:00, 125.41it/s, 63 steps of size 1.33e-02. acc. prob=0.93]


Bayesian inference complete.


### 7.1 Convergence Diagnostics

In [8]:
# Print convergence diagnostics
print_convergence(result, param_names)

# Print parameter summary table
print("\n" + "="*60)
print("PARAMETER POSTERIOR SUMMARY")
print("="*60)
print_parameter_table(param_names, nlsq_vals, result.posterior_samples)

Convergence Diagnostics
   Parameter     R-hat       ESS
--------------------------------------------------
         G_P    0.9918       105
         G_E    0.9912       110
        nu_0    0.9943       107
         E_a    1.0093        34 *
       V_act    1.0002        75 *
           T    0.9907        20 *
         G_D    0.9976        82 *
       k_d_D    1.0072        82 *

Divergences: 0
Convergence: CHECK REQUIRED

PARAMETER POSTERIOR SUMMARY

Parameter Comparison
       Param          NLSQ   Bayes (med)                    95% CI
------------------------------------------------------------------
         G_P            10     4.453e+08  [8.534e+07, 9.637e+08]
         G_E             5     4.744e+08  [1.912e+07, 9.764e+08]
        nu_0         1e+10     4.717e+13  [4.692e+12, 9.737e+13]
         E_a         8e+04     1.021e+05  [2.217e+04, 1.92e+05]
       V_act         1e-05      0.006216  [6.857e-05, 0.009733]
           T           300           331  [204.1, 470.5]
         

## 8. Posterior Predictive Analysis

Generate posterior predictive distribution to:
- Quantify uncertainty in flow curve predictions
- Identify parameter correlations
- Validate model against data

In [9]:
# Generate posterior predictive samples
print("Generating posterior predictive distribution...")
pred_draws = posterior_predictive_1d(
    model,
    gamma_dot_fit,
    result.posterior_samples,
    test_mode='flow_curve',
    n_draws=100
)

# Compute statistics
pred_mean = np.mean(pred_draws, axis=0)
pred_lo = np.percentile(pred_draws, 2.5, axis=0)
pred_hi = np.percentile(pred_draws, 97.5, axis=0)

# Plot posterior predictive
fig, ax = plt.subplots(figsize=(9, 6))

# Uncertainty bands (95% credible interval)
ax.fill_between(gamma_dot_fit, pred_lo, pred_hi, color='blue', alpha=0.15, label='95% CI')
ax.loglog(gamma_dot_fit, pred_mean, 'b-', lw=2, label='Posterior mean')

# Data
ax.loglog(gamma_dot, stress, 'ko', markersize=5, alpha=0.6, label='Data')

ax.set_xlabel("Shear rate (1/s)", fontsize=12)
ax.set_ylabel("Stress (Pa)", fontsize=12)
ax.set_title("HVM Flow Curve: Posterior Predictive", fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
save_figure(fig, get_output_dir('flow_curve'), 'hvm_flow_curve_posterior_predictive.png')
plt.close(fig)

Generating posterior predictive distribution...


Figure saved: /Users/b80985/Projects/rheojax/examples/outputs/hvm/flow_curve/figures/hvm_flow_curve_posterior_predictive.png


## 9. Viscosity Analysis

Compute effective viscosity η = σ/γ̇ to highlight:
- Yield stress (η → ∞ as γ̇ → 0)
- Shear thinning exponent
- Zero-shear viscosity η₀

In [10]:
# Compute viscosity from data and fit
eta_data = stress / gamma_dot
eta_fit = pred_mean / gamma_dot_fit

# Plot viscosity
fig, ax = plt.subplots(figsize=(9, 6))

# Uncertainty band from posterior
eta_lo = pred_lo / gamma_dot_fit
eta_hi = pred_hi / gamma_dot_fit
ax.fill_between(gamma_dot_fit, eta_lo, eta_hi, color='blue', alpha=0.15, label='95% CI')

# Mean and data
ax.loglog(gamma_dot_fit, eta_fit, 'b-', lw=2, label='Posterior mean')
ax.loglog(gamma_dot, eta_data, 'ko', markersize=5, alpha=0.6, label='Data')

ax.set_xlabel("Shear rate (1/s)", fontsize=12)
ax.set_ylabel("Viscosity (Pa.s)", fontsize=12)
ax.set_title("HVM: Apparent Viscosity", fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
save_figure(fig, get_output_dir('flow_curve'), 'hvm_flow_curve_viscosity.png')
plt.close(fig)

Figure saved: /Users/b80985/Projects/rheojax/examples/outputs/hvm/flow_curve/figures/hvm_flow_curve_viscosity.png


## 10. Save Results

Save all results for reproducibility:
- NLSQ parameters
- Posterior samples (HDF5)
- Figures (PNG)
- Metadata (config, diagnostics)

In [11]:
# Save results
output_dir = get_output_dir('flow_curve')
print(f"Saving results to: {output_dir}")

save_results(output_dir, model=model, result=result, param_names=param_names)

print("Results saved successfully.")

Saving results to: /Users/b80985/Projects/rheojax/examples/outputs/hvm/flow_curve
Results saved to /Users/b80985/Projects/rheojax/examples/outputs/hvm/flow_curve/
Results saved successfully.


## Summary

**Key Findings:**
1. HVM successfully captures yield stress and shear thinning in concentrated emulsion
2. Permanent network ($G_P$) dominates low-rate plateau
3. Dissociative network ($G_D$, $k_d^D$) controls high-rate viscosity
4. Bayesian inference provides robust uncertainty quantification

**Physical Insights:**
- $\sigma_E \to 0$ at steady state (natural state tracks deformation)
- Elastic plateau arises from permanent network
- Shear thinning from dissociative bond breaking

## Further Reading

- [HVM Model Reference](../../docs/source/models/hvm/hvm.rst) — Steady-state solution and protocol summary
- [HVM Knowledge Extraction](../../docs/source/models/hvm/hvm_knowledge.rst) — Parameter identifiability and cross-protocol validation

**References:**
1. Vernerey, F.J., Long, R. & Brighenti, R. (2017). *J. Mech. Phys. Solids*, 107, 1-20.

## Next Notebooks

- **Notebook 09**: Creep NLSQ + NUTS
- **Notebook 10**: Relaxation NLSQ + NUTS